## Q3 Clustering 

### Import libraries

In [233]:
%matplotlib inline
#!pip install geopandas
#!pip install geocoder
#!pip install folium 
import numpy as np
import pandas as pd
import requests
import io
import geocoder
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Libraries imported.


### Prepare for data table

In [235]:
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(source, header=0, flavor = 'bs4')[0]
df1 = df[df.Borough !='Not assigned']

df2=df1.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.reset_index(level=0, inplace=True)
geo=pd.read_csv("https://cocl.us/Geospatial_data")
geo.rename(columns={'Postal Code':'Postcode'}, inplace = True)
df3 = pd.merge(df2, geo, on='Postcode')
df3.rename(columns={'Neighbourhood':'Neighborhood'}, inplace = True)
df3

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [236]:
print('Final data has {} boroughs and {} neighborhoods.'.format(
        len(df3['Borough'].unique()),
        df3.shape[0]
    )
)

Final data has 11 boroughs and 103 neighborhoods.


In [237]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#latitude=43.653963
#longitude=-79.387207
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [247]:
# create map of Toronto using latitude and longitude values

map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map


### I am going to only select 'Downtown Toronto' to do the analysis.

In [248]:
data = pd.merge(df1, geo, on='Postcode')

Toronto_data = data[data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data.rename(columns={'Neighbourhood':'Neighborhood'}, inplace = True)

Toronto_data

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568
9,M5H,Downtown Toronto,King,43.650571,-79.384568


### let's visualize Toronto downtown with the neighborhoods in it

In [249]:
# create map of Toroto using latitude and longitude values

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], 
                                           Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(Toronto_map)  
    
Toronto_map


### Define Foursquare Credentials and Version


In [250]:
CLIENT_ID = '2H1VW2PPRXM2AHRPXETAL3PYEOXRBC3F4DXQOTOQEDUYOAPC' # your Foursquare ID
CLIENT_SECRET = '1KGSIJRGUENFBJ3AWEON3TOCMCXQOBJTZJTWXGPS3410PGT3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2H1VW2PPRXM2AHRPXETAL3PYEOXRBC3F4DXQOTOQEDUYOAPC
CLIENT_SECRET:1KGSIJRGUENFBJ3AWEON3TOCMCXQOBJTZJTWXGPS3410PGT3


### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [251]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [178]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [252]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto(boroughs that contain the word Toronto):

In [253]:

LIMIT = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [181]:
# type your answer here

Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude'])

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


### Let's check the size of the resulting dataframe

In [254]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2490, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [255]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,55,55,55,55,55,55
CN Tower,13,13,13,13,13,13
Cabbagetown,43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
Chinatown,100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89


In [256]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


### Analyze Each Neighborhood

In [257]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [258]:
Toronto_onehot.shape

(2490, 204)

In [196]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.010000,0.010000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.00000

In [259]:
Toronto_grouped.shape

(36, 204)

### Let's print each neighborhood along with the top 5 most common venues

In [260]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1  Thai Restaurant  0.04
2             Café  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Bathurst Quay----
              venue  freq
0  Airport Terminal  0.15
1   Airport Service  0.15
2    Airport Lounge  0.15
3             Plane  0.08
4      Airport Gate  0.08


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Steakhouse  0.04
3          Café  0.04
4   Cheese Shop  0.04


----CN Tower----
              venue  freq
0  Airport Terminal  0.15
1   Airport Service  0.15
2    Airport Lounge  0.15
3             Plane  0.08
4      Airport Gate  0.08


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2              Bakery  0.05
3         Pizza Place  0.05
4  Italian Restaurant  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.05
2        Burger Joint

### Let's put that into a pandas dataframe

In [190]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [261]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
1,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Farmers Market,Steakhouse,Restaurant,Pub,Bakery
3,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
4,Cabbagetown,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Café,Pizza Place,Pub,Liquor Store,Japanese Restaurant,Jewelry Store
5,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bubble Tea Shop,Café,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant
6,Chinatown,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
7,Christie,Grocery Store,Café,Park,Diner,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
8,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant
9,Commerce Court,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Bakery,Deli / Bodega


### Cluster Neighborhoods

In [262]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([3, 1, 4, 1, 4, 4, 3, 0, 3, 4, 4, 4, 3, 3, 3, 4, 4, 1, 1, 3, 3, 1, 1,
       4, 3, 2, 3, 1, 4, 4, 4, 4, 4, 4, 3, 4], dtype=int32)

In [210]:
neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
1,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Farmers Market,Steakhouse,Restaurant,Pub,Bakery
3,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
4,Cabbagetown,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Café,Pizza Place,Pub,Liquor Store,Japanese Restaurant,Jewelry Store
5,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bubble Tea Shop,Café,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant
6,Chinatown,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
7,Christie,Grocery Store,Café,Park,Diner,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
8,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant
9,Commerce Court,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Italian Restaurant,Seafood Restaurant,Gastropub,Bakery,Deli / Bodega


In [263]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Café,Breakfast Spot,Hotel,Bakery,Italian Restaurant,Gastropub,Park,Diner


In [271]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], 
                                  Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

####  We have 5 different clusters as below. We  can define their cluster names according to their neighbourhood

#### Cluster 1 Grocery store

In [265]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Diner,Convenience Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub


#### Cluster 2 Airport Terminal related

In [267]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
24,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
25,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
26,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
27,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
28,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport
29,Downtown Toronto,1,Airport Terminal,Airport Service,Airport Lounge,Plane,Airport Gate,Boat or Ferry,Sculpture Garden,Airport Food Court,Harbor / Marina,Airport


#### Cluster 3 Park

In [268]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


#### Cluster 4 Coffee or restaurant

In [269]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
3,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Lingerie Store,Pizza Place,Bar
8,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
9,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
10,Downtown Toronto,3,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Bakery,Burger Joint,Gym,Hotel,Sushi Restaurant
18,Downtown Toronto,3,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Theater,Bakery,Coffee Shop,Beer Bar,Beer Store
19,Downtown Toronto,3,Café,Bar,Japanese Restaurant,Bookstore,Restaurant,Theater,Bakery,Coffee Shop,Beer Bar,Beer Store
20,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
21,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant
22,Downtown Toronto,3,Café,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Farmers Market,Dumpling Restaurant


#### Cluster 5 Caffee shop

In [270]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4,
                     Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
1,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Restaurant
4,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Breakfast Spot,Hotel,Bakery,Italian Restaurant,Gastropub,Park,Diner
5,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Farmers Market,Steakhouse,Restaurant,Pub,Bakery
6,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bubble Tea Shop,Café,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant
11,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
12,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
13,Downtown Toronto,4,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Bakery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Brewery
14,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Steakhouse
15,Downtown Toronto,4,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Steakhouse


### End of page. Thank you!